### 0. Set up Python virtual environments with Snellius' JupyterHub

### 1. Run PRS-CSx

### 2. Matrices Preperation
### 2.1 Prepare the weight matrices W_eas & W_eur

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [18]:
# Change these paths to your own paths
path_EAS_var_w = '/home/scur0497/test/test_output/EAS_var_w.txt'
path_EUR_var_w = '/home/scur0497/test/test_output/EUR_var_w.txt'

# Read in the files and put them in dictionaries
EAS_var_w = pd.read_csv(path_EAS_var_w, sep = '\t', header = None) # read the file in as a panda dataframe; sep = '\t' --> deliminator is a tab character
EUR_var_w = pd.read_csv(path_EUR_var_w, sep = '\t', header = None)
EAS_dic = dict(zip(EAS_var_w[0], EAS_var_w[1])) # zip function: pairs each entry from column 0 with the corresponding entry in column 1 in the EAS_var_w dataframe
EUR_dic = dict(zip(EAS_var_w[0], EAS_var_w[1]))

# Find overlapping rsid variants and create a new dictionary with these variants and a tuple of values from both dictionaries
overlap_var = set(EAS_dic.keys()).intersection(set(EUR_dic.keys())) # convert keys (variants) from both dictionaries into sets; intersection method: identifies overlapping keys between two sets
overlap_var_list = list(overlap_var) # convert the set of overlapping keys into a list for later use
combined_dict = {key: (EAS_dic[key], EUR_dic[key]) for key in overlap_var} # create a new dictionary; keys = overlapping variants; values = tuple containing the corresponding weight values for that variant from EAS_dic and EUR_dic; *tuple = immutable data type that stores multiple items into one variable


# Write the overlapping variants to a txt file

# Change this path to your own path
path_overlap_risk_variants = '/home/scur0497/test/test_output/overlap_risk_variants.txt'

with open(path_overlap_risk_variants, 'w') as f: # open a file in the specified path for writing mode; context manaer 'with' --> ensure the file is properly closed after writing
    for key in overlap_var: # iterate through the set of overlapping variants
        f.write(f"{key}\n") # write out each variant followed by a new line character "\n" --> each variant written on a new line

In [19]:
# Prepare the variant weights matrices

pd.set_option('display.float_format', lambda x: '%.10f' % x) # display floating point numbers up to 10 decimal places; ensures that the numbers are shown with full precision

# W_eas


# W_eur


### 2.2 Prepare the genotype matrix X and phenotype matrix y in validation and testing datasets

In [20]:
# This path is same for everyone
path_data = "/projects/0/pgsr0673/scur0497/genotype_phenotype_1kgEAS.txt"

# Read in the file that has genotype and phenotype for each individual


# The file is prepared such that it has 504 rows (individuals) x 902 columns (variants + effect sizes); entries: first 901 columns: allele count for each variant; last column = effect size of that variant from PRS-CSx


[[2.         0.         0.         ... 0.         1.         0.04249645]
 [2.         0.         1.         ... 0.         0.         0.08080722]
 [1.         0.         2.         ... 1.         2.         0.06038308]
 ...
 [0.         0.         1.         ... 0.         1.         0.03393009]
 [1.         0.         2.         ... 0.         0.         0.04689794]
 [1.         0.         2.         ... 0.         1.         0.03195786]]
(504, 902)


In [21]:
# Separate genotype and phenotype information

# print the matrices and their shapes for inspection


In [22]:
# Split into validation and test datasets


# set a random seed for reproducibility


# specify the proportion of data to be used for validation


# calculate the number of samples based on the proportion


# set the validation indices


# set the test indices


# Extract the rows according to the indices for geno, phen, vali, and test respectively



### 3. Run regression to find weight parameters a_hat and b_hat

In [8]:
# Prepare the model input

# based on the equation y = a_hat * X @ Weas + b_hat * X @ Weur, calculate the weighted input data for the EAS population

# horizontally stack the weighted inputs for both population for model input
XW_vali = np.hstack((XWeas_vali, XWeur_vali)) 

# In essence, this block of code takes the validation data, 
# multiplies it by population-specific weight matrices, 
# and then concatenates the results side by side to form a combined input for the model.

In [9]:
# Fit the model



In [10]:
# Obtain the regression parameters
a_hat = model.coef_[0]
b_hat = model.coef_[1]
print(f"{a_hat =}")
print(f"{b_hat =}")

a_hat =0.9337471644122599
b_hat =-0.08808776795046623


### 4. Predict phenotype on validation and test dataset

In [11]:
# Make predictions on validation data



(201, 1)
(201,)


In [12]:
# Make the prediction on test data



### 5. Plot true values against predicted values

In [13]:
# create a blank canvas 10 in. wide x 5 inch. tall


# Compute global minimum and maximum for x and y axes for setting the x & y ranges
min_true = min(min(y_vali), min(y_test))
max_true = max(max(y_vali), max(y_test))
min_pred = min(min(y_hat_vali), min(y_hat))
max_pred = max(max(y_hat_vali), max(y_hat))

# For the validation cohort
plt.subplot(1, 2, 1) # (# rows, # columns, index of this plot)
plt.scatter(y_vali, y_hat_vali, alpha = 0.5) # alpha sets the transparency of the dot markers
plt.title('Validation Dataset')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.plot([min_true, max_true], [min_pred, max_pred], 'k', linestyle='dashed') # draw a line that shows perfect correlation between true and predicted values

# For the test cohort


# Save the figure & inspect it!



### 6. Evaluate using deviance-based R^2

In [23]:
# Calculate the deviance based on the y_hat for the test data (= residual deviance = total squared difference between the true output values and the predicted output values)


# Print the deviance


In [15]:
# Calculate null deviance = total squared difference between the true output values and their mean


# Print the null deviance



deviance_null = 0.19363495614719278


In [24]:
# Calculate deviance-based R2

# Print the R2

